# Orbital Maneuvers and Transfers

This notebook covers orbital maneuvers and transfer trajectories - essential for satellite operations and mission planning.

**Topics covered:**
- Hohmann transfers (most fuel-efficient)
- Bielliptic transfers (for large radius changes)
- Plane change maneuvers
- Lambert's problem (solving for transfer orbits)
- Interplanetary transfers
- Delta-v budgets

**Prerequisites:** Understanding of orbital mechanics from 02_orbital_mechanics.ipynb

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astrora import Orbit, _core, bodies
from astrora.maneuvers import bielliptic_transfer, hohmann_transfer

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

print("✓ Imports successful!")

## 1. Hohmann Transfer

The **Hohmann transfer** is the most fuel-efficient two-impulse maneuver for transferring between two circular coplanar orbits.

### Theory

For a transfer from radius $r_1$ to radius $r_2$:

1. **First impulse** at $r_1$: Raise apoapsis to $r_2$
2. **Coast** along transfer ellipse
3. **Second impulse** at $r_2$: Circularize orbit

Total transfer time: **half the period of transfer ellipse**

In [ ]:
# LEO to GEO transfer
r_leo = 6778e3  # 400 km altitude (m)
r_geo = 42164e3  # geostationary radius (m)
mu = bodies.Earth.mu.to_value('m^3/s^2')

# Calculate Hohmann transfer
dv1, dv2, transfer_time = hohmann_transfer(r_leo, r_geo, mu)

print("LEO to GEO Hohmann Transfer")
print("="*60)
print(f"Initial orbit (LEO):")
print(f"  Radius: {r_leo/1000:.0f} km")
print(f"  Altitude: {(r_leo - 6371e3)/1000:.0f} km")
print(f"  Velocity: {np.sqrt(mu/r_leo):.2f} m/s")
print(f"\nFinal orbit (GEO):")
print(f"  Radius: {r_geo/1000:.0f} km")
print(f"  Altitude: {(r_geo - 6371e3)/1000:.0f} km")
print(f"  Velocity: {np.sqrt(mu/r_geo):.2f} m/s")
print(f"\nManeuver Requirements:")
print(f"  ΔV₁ (at LEO perigee): {dv1:.2f} m/s = {dv1/1000:.3f} km/s")
print(f"  ΔV₂ (at GEO apogee): {dv2:.2f} m/s = {dv2/1000:.3f} km/s")
print(f"  Total ΔV: {(dv1 + dv2)/1000:.3f} km/s")
print(f"  Transfer time: {transfer_time/3600:.2f} hours")
print("="*60)

# Calculate transfer orbit properties
a_transfer = (r_leo + r_geo) / 2
ecc_transfer = (r_geo - r_leo) / (r_geo + r_leo)
period_transfer = 2 * np.pi * np.sqrt(a_transfer**3 / mu)

print(f"\nTransfer Orbit Properties:")
print(f"  Semi-major axis: {a_transfer/1000:.0f} km")
print(f"  Eccentricity: {ecc_transfer:.4f}")
print(f"  Perigee: {r_leo/1000:.0f} km")
print(f"  Apogee: {r_geo/1000:.0f} km")
print(f"  Period: {period_transfer/3600:.2f} hours")

### Visualize Hohmann Transfer

In [ ]:
# Create orbits
leo_orbit = Orbit.from_classical(
    bodies.Earth, a=r_leo * u.m, ecc=0.0 * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

geo_orbit = Orbit.from_classical(
    bodies.Earth, a=r_geo * u.m, ecc=0.0 * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

transfer_orbit = Orbit.from_classical(
    bodies.Earth, a=a_transfer * u.m, ecc=ecc_transfer * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

# Plot
fig, ax = plt.subplots(figsize=(14, 14))

# Earth
earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth', zorder=5)
ax.add_patch(earth_circle)

# Helper function to get orbit xy coordinates
def get_orbit_xy(orbit, n_points=100, half_orbit=False):
    if half_orbit:
        theta = np.linspace(0, np.pi, n_points)
    else:
        theta = np.linspace(0, 2*np.pi, n_points)
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            orbit.attractor, orbit.a, orbit.ecc, orbit.inc,
            orbit.raan, orbit.argp, angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)
    return positions[:, 0], positions[:, 1]

# Plot orbits
x_leo, y_leo = get_orbit_xy(leo_orbit)
x_geo, y_geo = get_orbit_xy(geo_orbit)
x_transfer, y_transfer = get_orbit_xy(transfer_orbit, half_orbit=True)

ax.plot(x_leo, y_leo, 'g-', linewidth=3, label='LEO (Initial)', zorder=3)
ax.plot(x_geo, y_geo, 'b-', linewidth=3, label='GEO (Target)', zorder=3)
ax.plot(x_transfer, y_transfer, 'r--', linewidth=3, label='Transfer Orbit', zorder=4)

# Mark burn locations
ax.scatter([r_leo/1000], [0], color='red', s=400, marker='*',
           zorder=6, edgecolors='black', linewidths=2, label=f'ΔV₁ = {dv1/1000:.2f} km/s')
ax.scatter([-r_geo/1000], [0], color='orange', s=400, marker='*',
           zorder=6, edgecolors='black', linewidths=2, label=f'ΔV₂ = {dv2/1000:.2f} km/s')

# Add annotations
ax.annotate('Start\n(Burn 1)', xy=(r_leo/1000, 0), xytext=(r_leo/1000 + 5000, 8000),
            arrowprops=dict(arrowstyle='->', lw=2, color='red'),
            fontsize=12, fontweight='bold', color='red')
ax.annotate('End\n(Burn 2)', xy=(-r_geo/1000, 0), xytext=(-r_geo/1000 - 5000, -8000),
            arrowprops=dict(arrowstyle='->', lw=2, color='orange'),
            fontsize=12, fontweight='bold', color='orange')

ax.set_xlabel('X (km)', fontsize=13)
ax.set_ylabel('Y (km)', fontsize=13)
ax.set_title('Hohmann Transfer: LEO → GEO', fontsize=16, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.set_xlim(-50000, 50000)
ax.set_ylim(-50000, 50000)
plt.tight_layout()
plt.show()

## 2. Bielliptic Transfer

For **large** radius ratio changes ($r_2/r_1 > 11.94$), a **bielliptic transfer** can be more fuel-efficient than Hohmann!

Uses **three impulses**:
1. Raise apoapsis to intermediate radius $r_b$
2. At $r_b$, raise periapsis to final radius $r_2$
3. At $r_2$, circularize

Trade-off: Lower ΔV but **much longer transfer time**

In [ ]:
# LEO to very high orbit (lunar distance scale)
r1 = 6778e3   # LEO
r2 = 100000e3  # 100,000 km (much higher than GEO)
rb = 200000e3  # Intermediate apoapsis (bielliptic)

# Hohmann transfer
dv1_h, dv2_h, time_h = hohmann_transfer(r1, r2, mu)
total_dv_hohmann = dv1_h + dv2_h

# Bielliptic transfer
dv1_b, dv2_b, dv3_b, time_b = bielliptic_transfer(r1, rb, r2, mu)
total_dv_bielliptic = dv1_b + dv2_b + dv3_b

print("Transfer Comparison: LEO to High Orbit (100,000 km)")
print("="*70)
print(f"\nHohmann Transfer (2 burns):")
print(f"  ΔV₁: {dv1_h/1000:.3f} km/s")
print(f"  ΔV₂: {dv2_h/1000:.3f} km/s")
print(f"  Total ΔV: {total_dv_hohmann/1000:.3f} km/s")
print(f"  Time: {time_h/3600:.2f} hours ({time_h/86400:.2f} days)")

print(f"\nBielliptic Transfer (3 burns, rb={rb/1000:.0f} km):")
print(f"  ΔV₁: {dv1_b/1000:.3f} km/s")
print(f"  ΔV₂: {dv2_b/1000:.3f} km/s")
print(f"  ΔV₃: {dv3_b/1000:.3f} km/s")
print(f"  Total ΔV: {total_dv_bielliptic/1000:.3f} km/s")
print(f"  Time: {time_b/3600:.2f} hours ({time_b/86400:.2f} days)")

dv_savings = (total_dv_hohmann - total_dv_bielliptic)
time_penalty = (time_b - time_h)

print(f"\nComparison:")
print(f"  ΔV savings (bielliptic): {dv_savings:.2f} m/s ({dv_savings/total_dv_hohmann*100:.1f}%)")
print(f"  Time penalty: {time_penalty/86400:.2f} days")
print("="*70)

if total_dv_bielliptic < total_dv_hohmann:
    print("\n✓ Bielliptic is MORE efficient for this large radius change!")
else:
    print("\n✗ Hohmann is still more efficient for this radius change.")

## 3. Plane Change Maneuvers

Changing orbital inclination is **very expensive** in terms of ΔV.

For a simple plane change:
$$\Delta V = 2v \sin(\Delta i / 2)$$

where $v$ is orbital velocity and $\Delta i$ is the inclination change.

In [ ]:
# Calculate plane change ΔV for different scenarios
scenarios = [
    {'name': 'LEO', 'r': 6778e3, 'di': [1, 5, 10, 30, 60, 90]},
    {'name': 'GEO', 'r': 42164e3, 'di': [1, 5, 10, 30, 60, 90]},
]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

for ax, scenario in zip(axes, scenarios):
    r = scenario['r']
    v = np.sqrt(mu / r)  # Circular orbit velocity

    delta_vs = []
    for di in scenario['di']:
        dv = 2 * v * np.sin(np.radians(di) / 2)
        delta_vs.append(dv / 1000)  # km/s

    ax.bar(range(len(scenario['di'])), delta_vs, color='red', edgecolor='black', linewidth=2)
    ax.set_xticks(range(len(scenario['di'])))
    ax.set_xticklabels([f"{di}°" for di in scenario['di']])
    ax.set_xlabel('Inclination Change', fontsize=12)
    ax.set_ylabel('ΔV (km/s)', fontsize=12)
    ax.set_title(f'Plane Change Cost at {scenario["name"]}', fontsize=13, fontweight='bold')
    ax.grid(True, axis='y', alpha=0.3)

    # Add values on bars
    for i, dv in enumerate(delta_vs):
        ax.text(i, dv + 0.1, f'{dv:.2f}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("Key Insight: Plane changes are MUCH cheaper at higher altitudes!")
print(f"  90° plane change at LEO: {2 * np.sqrt(mu/6778e3)/1000:.2f} km/s")
print(f"  90° plane change at GEO: {2 * np.sqrt(mu/42164e3)/1000:.2f} km/s")
print("\nStrategy: Combine plane change with apoapsis raise for savings!")

## 4. Lambert's Problem

**Lambert's problem**: Find the orbit connecting two position vectors in a given time.

Essential for:
- Interplanetary transfers
- Rendezvous planning
- Launch window optimization

Astrora uses a high-performance Rust implementation of Izzo's Lambert solver.

In [ ]:
# Simple Earth orbit transfer using Lambert's problem
# Position 1: LEO at (r, 0, 0)
r1 = np.array([7000e3, 0.0, 0.0])  # m

# Position 2: Different location after 45° around Earth
angle = np.radians(45)
r_target = 20000e3  # Target at 20,000 km
r2 = np.array([r_target * np.cos(angle), r_target * np.sin(angle), 0.0])

# Time of flight: 2 hours
tof = 2 * 3600.0  # seconds

# Solve Lambert's problem
v1, v2 = _core.lambert_izzo(
    r1, r2, tof, mu,
    prograde=True,  # Short way
    low_path=True,  # Low-energy solution
    max_iter=100,
    rtol=1e-8
)

print("Lambert's Problem Solution")
print("="*60)
print(f"Starting position: {r1/1000} km")
print(f"Target position: {r2/1000} km")
print(f"Time of flight: {tof/3600:.2f} hours")
print(f"\nRequired velocities:")
print(f"  Initial velocity: {v1} m/s")
print(f"  Final velocity: {v2} m/s")
print(f"  Initial speed: {np.linalg.norm(v1)/1000:.3f} km/s")
print(f"  Final speed: {np.linalg.norm(v2)/1000:.3f} km/s")

# Calculate transfer orbit properties
coe = _core.rv_to_coe(r1, v1, mu)
print(f"\nTransfer orbit:")
print(f"  Semi-major axis: {coe[0]/1000:.2f} km")
print(f"  Eccentricity: {coe[1]:.4f}")
print(f"  Inclination: {np.rad2deg(coe[2]):.4f}°")
print("="*60)

### Visualize Lambert Solution

In [ ]:
# Create transfer orbit from Lambert solution
transfer_lambert = Orbit.from_vectors(bodies.Earth, r1 * u.m, v1 * u.m/u.s)

# Propagate to verify it reaches r2
propagated = transfer_lambert.propagate(tof * u.s)
r2_check = propagated.r.to_value('m')

print(f"Verification:")
print(f"  Target position: {r2/1000}")
print(f"  Reached position: {r2_check/1000}")
print(f"  Error: {np.linalg.norm(r2 - r2_check):.2f} m")

# Plot the transfer
fig, ax = plt.subplots(figsize=(12, 12))

# Earth
earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth', zorder=5)
ax.add_patch(earth_circle)

# Propagate transfer orbit
times = np.linspace(0, tof, 100)
positions = []
for t in times:
    prop = transfer_lambert.propagate(t * u.s)
    positions.append(prop.r.to_value('km'))
positions = np.array(positions)

# Plot transfer trajectory
ax.plot(positions[:, 0], positions[:, 1], 'r-', linewidth=3, label='Transfer Orbit')

# Mark start and end positions
ax.scatter([r1[0]/1000], [r1[1]/1000], color='green', s=300, marker='o',
           zorder=6, edgecolors='black', linewidths=2, label='Start')
ax.scatter([r2[0]/1000], [r2[1]/1000], color='red', s=300, marker='s',
           zorder=6, edgecolors='black', linewidths=2, label='Target')

# Add velocity vectors
scale = 3000  # Scale for visibility
ax.arrow(r1[0]/1000, r1[1]/1000, v1[0]*scale/1000, v1[1]*scale/1000,
         head_width=500, head_length=700, fc='green', ec='darkgreen', linewidth=2)
ax.arrow(r2[0]/1000, r2[1]/1000, v2[0]*scale/1000, v2[1]*scale/1000,
         head_width=500, head_length=700, fc='red', ec='darkred', linewidth=2)

ax.set_xlabel('X (km)', fontsize=12)
ax.set_ylabel('Y (km)', fontsize=12)
ax.set_title(f'Lambert\'s Problem Solution (TOF = {tof/3600:.1f} hours)',
             fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

## 5. Interplanetary Transfers

Let's plan an Earth-Mars transfer using Lambert's problem with simplified circular orbits.

In [ ]:
# Simplified planetary parameters (circular orbits)
r_earth_orbit = 1.0 * 149597870700  # 1 AU in meters
r_mars_orbit = 1.524 * 149597870700  # ~1.52 AU
mu_sun = bodies.Sun.mu.to_value('m^3/s^2')

# Earth and Mars positions (simplified: coplanar circular orbits)
r_earth = np.array([r_earth_orbit, 0.0, 0.0])

# Mars at optimal Hohmann transfer angle
# For Hohmann, Mars should be 180° - θ ahead, where θ is Earth's travel angle
# Simplified: Use Mars at specific angle for demonstration
mars_angle = np.radians(60)  # Example angle
r_mars = np.array([r_mars_orbit * np.cos(mars_angle),
                   r_mars_orbit * np.sin(mars_angle), 0.0])

# Time of flight: typical Earth-Mars transfer ~8 months
tof_mars = 200 * 86400  # 200 days in seconds

# Solve Lambert's problem
v_earth_dep, v_mars_arr = _core.lambert_izzo(
    r_earth, r_mars, tof_mars, mu_sun,
    prograde=True, low_path=True,
    max_iter=100, rtol=1e-8
)

# Calculate delta-v requirements
v_earth_circ = np.sqrt(mu_sun / r_earth_orbit)
v_mars_circ = np.sqrt(mu_sun / r_mars_orbit)

# Earth departure
v_earth_orbit_vec = np.array([0, v_earth_circ, 0])
dv_departure = np.linalg.norm(v_earth_dep - v_earth_orbit_vec)

# Mars arrival
v_mars_orbit_vec = np.array([-v_mars_circ * np.sin(mars_angle),
                              v_mars_circ * np.cos(mars_angle), 0])
dv_arrival = np.linalg.norm(v_mars_arr - v_mars_orbit_vec)

# C3 characteristic energy (for launch vehicle)
v_inf_earth = np.linalg.norm(v_earth_dep - v_earth_orbit_vec)
c3 = v_inf_earth**2 / 1e6  # km²/s²

print("Earth-Mars Transfer Mission")
print("="*70)
print(f"Departure from Earth orbit (1 AU)")
print(f"Arrival at Mars orbit (1.524 AU)")
print(f"Time of flight: {tof_mars/86400:.0f} days ({tof_mars/86400/30:.1f} months)")
print(f"\nDelta-V Requirements:")
print(f"  Departure from Earth orbit: {dv_departure/1000:.3f} km/s")
print(f"  Arrival at Mars orbit: {dv_arrival/1000:.3f} km/s")
print(f"  Total heliocentric ΔV: {(dv_departure + dv_arrival)/1000:.3f} km/s")
print(f"\nLaunch Energy:")
print(f"  C3 characteristic energy: {c3:.2f} km²/s²")
print(f"  V∞ (hyperbolic excess): {v_inf_earth/1000:.3f} km/s")
print("="*70)

# Launch vehicle compatibility
launch_vehicles = [
    ('Falcon 9', 23),
    ('Atlas V 551', 47),
    ('Delta IV Heavy', 66),
    ('Falcon Heavy', 98),
]

print(f"\nLaunch Vehicle Compatibility (C3 = {c3:.1f} km²/s²):")
for vehicle, max_c3 in launch_vehicles:
    status = "✓" if c3 <= max_c3 else "✗"
    print(f"  {status} {vehicle:20s} (max C3 = {max_c3} km²/s²)")

## 6. Delta-V Budget Analysis

Complete mission delta-v budget for LEO to Mars surface:

In [ ]:
# Mission phases and their delta-v costs
mission_phases = [
    ('Launch to LEO', 9400),  # Includes gravity/drag losses
    ('LEO to Earth escape', 3200),  # Trans-Mars injection
    ('Mars orbit insertion', 2000),  # Capture into Mars orbit
    ('Mars descent & landing', 600),  # With aerobraking
]

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
phases = [p[0] for p in mission_phases]
dvs = [p[1] for p in mission_phases]
colors = ['red', 'orange', 'blue', 'green']

bars = ax1.barh(phases, dvs, color=colors, edgecolor='black', linewidth=2)
ax1.set_xlabel('ΔV (m/s)', fontsize=12)
ax1.set_title('Earth-Mars Mission ΔV Budget', fontsize=14, fontweight='bold')
ax1.grid(True, axis='x', alpha=0.3)

# Add values on bars
for i, (bar, dv) in enumerate(zip(bars, dvs)):
    ax1.text(dv + 200, i, f'{dv} m/s', va='center', fontsize=11, fontweight='bold')

# Cumulative delta-v
cumulative = np.cumsum(dvs)
ax2.plot(range(len(phases)), cumulative, 'o-', linewidth=3, markersize=10, color='red')
ax2.set_xticks(range(len(phases)))
ax2.set_xticklabels(range(1, len(phases)+1))
ax2.set_xlabel('Mission Phase', fontsize=12)
ax2.set_ylabel('Cumulative ΔV (m/s)', fontsize=12)
ax2.set_title('Cumulative ΔV Budget', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Add annotations
for i, (phase, cum_dv) in enumerate(zip(phases, cumulative)):
    ax2.annotate(f'{cum_dv} m/s', (i, cum_dv), xytext=(10, 10),
                textcoords='offset points', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

total_dv = sum(dvs)
print(f"Total mission ΔV: {total_dv} m/s = {total_dv/1000:.1f} km/s")
print(f"\nMass ratio (Tsiolkovsky): exp(ΔV/ve)")
print(f"  With ve=4500 m/s (chemical): {np.exp(total_dv/4500):.1f}")
print(f"  Initial mass = {np.exp(total_dv/4500):.1f} × payload mass")

## Summary

In this notebook, you learned:

✅ Hohmann transfers - most efficient two-impulse maneuver

✅ Bielliptic transfers - better for very large radius changes

✅ Plane change maneuvers - expensive but necessary

✅ Lambert's problem - solving for transfer orbits

✅ Interplanetary mission planning

✅ Complete delta-v budget analysis

### Next Steps

Continue with:
- **04_visualization_plotting.ipynb** - Advanced plotting and animation
- **05_advanced_techniques.ipynb** - Monte Carlo, perturbations, performance optimization